# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [11]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')
df.drop(['Unnamed: 0', 'id_cliente', 'data_ref'], axis=1, inplace=True)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
# 1

X = df.drop('renda', axis=1)
y = df['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

df_test = pd.concat([X_test, y_test], axis=1)

In [5]:
#2 melhor modelo = alpha = 0

modelo = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(qtd_filhos) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + C(qt_pessoas_residencia)'

md = smf.ols(modelo, data = df_test)
reg = md.fit_regularized(method = 'elastic_net'
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = 0)

reg.summary()

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     24.05
Date:                Fri, 12 Jan 2024   Prob (F-statistic):          3.69e-127
Time:                        18:25:26   Log-Likelihood:                -4489.9
No. Observations:                3750   AIC:                             9046.
Df Residuals:                    3718   BIC:                             9251.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.2996      0.308     23.681      0.000       6.695       7.904
C(sexo)[T.M]                            0.6984      0.031     22.856      0.000       0.638       0.758
C(posse_de_veiculo)[T.True]            -0.0142      0.029     -0.488      0.626      -0.071       0.043
C(posse_de_imovel)[T.True]              0.1154      0.029      3.983      0.000       0.059       0.172
C(qtd_filhos)[T.1]                     -0.7744      0.257     -3.018      0.003      -1.278      -0.271
C(qtd_filhos)[T.2]                     -0.9500      0.498     -1.908      0.057      -1.926       0.026
C(qtd_filhos)[T.3]                     -1.3039      0.929     -1.403      0.161      -3.126       0.518
C(qtd_filhos)[T.4]                      0.4690      0.208      2.257      0.024       0.062       0.876
C(qtd_filhos)[T.14]                    -0.4477      0.404     -1.109      0.268      -1.239       0.344
C(tipo_renda)[T.Bolsista]               0.2717      0.466      0.583      0.560      -0.642       1.185
C(tipo_renda)[T.Empresário]             0.0078      0.033      0.235      0.814      -0.057       0.073
C(tipo_renda)[T.Pensionista]           -0.2048      0.040     -5.114      0.000      -0.283      -0.126
C(tipo_renda)[T.Servidor público]       0.1824      0.047      3.878      0.000       0.090       0.275
C(educacao)[T.Pós graduação]           -0.2363      0.305     -0.776      0.438      -0.833       0.361
C(educacao)[T.Secundário]              -0.0218      0.141     -0.154      0.878      -0.299       0.256
C(educacao)[T.Superior completo]        0.0656      0.142      0.461      0.645      -0.213       0.345
C(educacao)[T.Superior incompleto]      0.0088      0.155      0.057      0.955      -0.296       0.313
C(estado_civil)[T.Separado]             0.6833      0.256      2.667      0.008       0.181       1.185
C(estado_civil)[T.Solteiro]             0.5300      0.253      2.096      0.036       0.034       1.026
C(estado_civil)[T.União]               -0.1365      0.051     -2.656      0.008      -0.237      -0.036
C(estado_civil)[T.Viúvo]                0.6423      0.260      2.466      0.014       0.132       1.153
C(tipo_residencia)[T.Casa]              0.0441      0.108      0.410      0.682      -0.167       0.255
C(tipo_residencia)[T.Com os pais]      -0.1695      0.122     -1.388      0.165      -0.409       0.070
C(tipo_residencia)[T.Comunitário]      -0.0895      0.256     -0.349      0.727      -0.592       0.413
C(tipo_residencia)[T.Estúdio]          -0.0130      0.218     -0.060      0.952      -0.441       0.414
C(tipo_residencia)[T.Governamental]    -0.0167      0.132     -0.126      0.900      -0.276       0.243
C(qt_pessoas_residencia)[T.2.0]         0.6424      0.253      2.54

In [6]:
#3 melhor modelo => alpha = 0

modelo2 = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(qtd_filhos) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + C(qt_pessoas_residencia)'

md2 = smf.ols(modelo2, data = df_test)
reg2 = md2.fit_regularized(method = 'elastic_net'
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0)

reg2.summary()

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     24.05
Date:                Fri, 12 Jan 2024   Prob (F-statistic):          3.69e-127
Time:                        18:25:27   Log-Likelihood:                -4489.9
No. Observations:                3750   AIC:                             9046.
Df Residuals:                    3718   BIC:                             9251.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.2996      0.308     23.681      0.000       6.695       7.904
C(sexo)[T.M]                            0.6984      0.031     22.856      0.000       0.638       0.758
C(posse_de_veiculo)[T.True]            -0.0142      0.029     -0.488      0.626      -0.071       0.043
C(posse_de_imovel)[T.True]              0.1154      0.029      3.983      0.000       0.059       0.172
C(qtd_filhos)[T.1]                     -0.7744      0.257     -3.018      0.003      -1.278      -0.271
C(qtd_filhos)[T.2]                     -0.9500      0.498     -1.908      0.057      -1.926       0.026
C(qtd_filhos)[T.3]                     -1.3039      0.929     -1.403      0.161      -3.126       0.518
C(qtd_filhos)[T.4]                      0.4690      0.208      2.257      0.024       0.062       0.876
C(qtd_filhos)[T.14]                    -0.4477      0.404     -1.109      0.268      -1.239       0.344
C(tipo_renda)[T.Bolsista]               0.2717      0.466      0.583      0.560      -0.642       1.185
C(tipo_renda)[T.Empresário]             0.0078      0.033      0.235      0.814      -0.057       0.073
C(tipo_renda)[T.Pensionista]           -0.2048      0.040     -5.114      0.000      -0.283      -0.126
C(tipo_renda)[T.Servidor público]       0.1824      0.047      3.878      0.000       0.090       0.275
C(educacao)[T.Pós graduação]           -0.2363      0.305     -0.776      0.438      -0.833       0.361
C(educacao)[T.Secundário]              -0.0218      0.141     -0.154      0.878      -0.299       0.256
C(educacao)[T.Superior completo]        0.0656      0.142      0.461      0.645      -0.213       0.345
C(educacao)[T.Superior incompleto]      0.0088      0.155      0.057      0.955      -0.296       0.313
C(estado_civil)[T.Separado]             0.6833      0.256      2.667      0.008       0.181       1.185
C(estado_civil)[T.Solteiro]             0.5300      0.253      2.096      0.036       0.034       1.026
C(estado_civil)[T.União]               -0.1365      0.051     -2.656      0.008      -0.237      -0.036
C(estado_civil)[T.Viúvo]                0.6423      0.260      2.466      0.014       0.132       1.153
C(tipo_residencia)[T.Casa]              0.0441      0.108      0.410      0.682      -0.167       0.255
C(tipo_residencia)[T.Com os pais]      -0.1695      0.122     -1.388      0.165      -0.409       0.070
C(tipo_residencia)[T.Comunitário]      -0.0895      0.256     -0.349      0.727      -0.592       0.413
C(tipo_residencia)[T.Estúdio]          -0.0130      0.218     -0.060      0.952      -0.441       0.414
C(tipo_residencia)[T.Governamental]    -0.0167      0.132     -0.126      0.900      -0.276       0.243
C(qt_pessoas_residencia)[T.2.0]         0.6424      0.253      2.54

In [7]:
# transformei as variáveis em numéricas
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Converti valore booleanos em numéricos
X_train = X_train * 1
X_test = X_test * 1

# Tratei NaNs em 'tempo_emprego'
tempo_emprego_media = X_train['tempo_emprego'].mean()
X_train['tempo_emprego'].fillna(tempo_emprego_media, inplace=True)
X_test['tempo_emprego'].fillna(tempo_emprego_media, inplace=True)

In [8]:
#4)

def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_test, y_test)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 9.97266e-127
#############
['tempo_emprego']
Add  sexo_F                         with p-value 3.044e-124
#############
['tempo_emprego', 'sexo_F']
Add  sexo_M                         with p-value 1.07631e-142
#############
['tempo_emprego', 'sexo_F', 'sexo_M']
Add  tipo_renda_Pensionista         with p-value 3.39093e-05
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista']
Add  idade                          with p-value 0.000379156
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista', 'idade']
Add  posse_de_imovel                with p-value 0.00473779
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista', 'idade', 'posse_de_imovel']
Add  educacao_Superior completo     with p-value 0.00683166
#############
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Pensionista', 'idade', 'posse_de_imovel', 'educacao_Superior completo']
Add  estado_civil_União             with 

In [9]:
modelo3 = sm.OLS(y_test, sm.add_constant(X_test[variaveis])).fit()

modelo3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     174.8
Date:                Fri, 12 Jan 2024   Prob (F-statistic):          2.11e-251
Time:                        18:25:34   Log-Likelihood:                -38523.
No. Observations:                3750   AIC:                         7.706e+04
Df Residuals:                    3741   BIC:                         7.712e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         19.5049    392.471      0.050      0.960    -749.973     788.983
tempo_emprego                563.2130     20.277     27.776      0.000     523.458     602.968
sexo_F                     -3005.9338    245.502    -12.244      0.000   -3487.264   -2524.603
sexo_M                      3025.4387    220.462     13.723      0.000    2593.201    3457.676
tipo_renda_Pensionista     -2000.4924    401.552     -4.982      0.000   -2787.774   -1213.210
idade                         48.0097     13.978      3.435      0.001      20.604      75.416
posse_de_imovel              696.7511    247.673      2.813      0.005     211.163    1182.339
educacao_Superior completo   605.3105    240.496      2.517      0.012     133.795    1076.826
estado_civil_União         -1018.3453    440.960     -2.309      0.021   -1882.892    -153.799
tipo_renda_Empresário        596.1660    283.313      2.104      0.035      40.703    1151.629
==============================================================================
Omnibus:                     5076.401   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2149213.201
Skew:                           7.428   Prob(JB):                         0.00
Kurtosis:                     119.337   Cond. No.                     1.54e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.33e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

5) aparentemente o modelo melhor é do stepwise, considerando o seu R² maior e ser mais simples


In [22]:
modelo_arvore = DecisionTreeRegressor()
modelo_arvore.fit(X_train, y_train)
y_pred_arvore = modelo_arvore.predict(X_test)
r2_arvore = r2_score(y_test, y_pred_arvore)

print(f"R² para Árvore de Regressão: {r2_arvore}")

#R² pra arvore de regressão ficou melhor

R² para Árvore de Regressão: 0.3048010836400066
